In [ ]:
"""
Date: 1/05/2025
Author: Asserewou Etekpo
Tpoic: Populate MySQL Database with data from an csv file with Multiple sheets using Python

"""

In [1]:
import pandas as pd
from datetime import datetime
import mysql.connector
from mysql.connector import Error

# Step 1: Read the Excel file into pandas DataFrames
file_path = 'C:/Users/honor/OneDrive/Documents/CreditCardProject/data/splitsubfraudtest.xlsx'


excel_data = pd.read_excel(file_path, sheet_name=None)  # Read all sheets

# Step 2: Assign each sheet to a DataFrame

df_Location = excel_data['sheet_Location']

df_Cardholder = excel_data['sheet_Cardholder']

df_Merchant = excel_data['sheet_Merchant']

df_Transaction = excel_data['sheet_Transaction']



In [ ]:
# Lets check for duplicates on the PK (zip) in df_Location dataframe

df_Location.duplicated() # We have some duplicates in the dataframe df_Location

In [10]:
# Lets drop duplicates in df_Location based on the 'zip' column

df_Location_unique = df_Location.drop_duplicates(subset="zip", keep="first")


In [ ]:
# Check for duplicates in the 'zip' column
duplicate_rows = df_Location_unique[df_Location_unique.duplicated(subset="zip", keep=False)]
print("Duplicate rows:")
print(duplicate_rows)

In [ ]:
# Lets check for duplicates on the PK (cc_num) in df_Cardholder dataframe

df_Cardholder.duplicated() # We have some duplicates in the dataframe df_Cardholder

In [12]:
# Lets drop duplicates in df_Cardholder based on the 'cc_num' column

df_Cardholder_unique = df_Cardholder.drop_duplicates(subset="cc_num", keep="first")


In [ ]:
# Check for duplicates in the 'cc_num' column
duplicate_rows1 = df_Cardholder_unique[df_Cardholder_unique.duplicated(subset="cc_num", keep=False)]
print("Duplicate rows:")
print(duplicate_rows1)

In [ ]:
df_Cardholder_unique.sample()

In [15]:
# Lets drop duplicates in df_Merchant based on the 'merchant' column

df_Merchant_unique = df_Merchant.drop_duplicates(subset="merchant", keep="first")

In [ ]:
# Check for duplicates in the 'merchant' column
duplicate_rows2 = df_Merchant_unique[df_Merchant_unique.duplicated(subset="merchant", keep=False)]
print("Duplicate rows:")
print(duplicate_rows2)

In [ ]:
df_Merchant_unique.sample()

In [18]:
# Lets drop duplicates in df_Transaction based on the 'trans_num' column

df_Transaction_unique = df_Transaction.drop_duplicates(subset="trans_num", keep="first")

In [ ]:
# Check for duplicates in the 'trans_num' column
duplicate_rows3 = df_Transaction_unique[df_Transaction_unique.duplicated(subset="trans_num", keep=False)]
print("Duplicate rows:")
print(duplicate_rows3)

In [ ]:
df_Transaction_unique.sample(1)

# Duplicates have been removed on all Primary Key's in all dataframes

In [ ]:
# Lets check for the format of the column dob.

df_Cardholder_unique["dob"]

In [ ]:
# Let's convert the 'dob' column to the correct format (YYYY-MM-DD) in df_Cardholder

df_Cardholder_unique['dob'] = pd.to_datetime(df_Cardholder_unique['dob'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [ ]:
# Convert the column to datetime type (if not already)

df_Transaction_unique['trans_date_trans_time'] = pd.to_datetime(df_Transaction_unique['trans_date_trans_time'])


In [ ]:
# Convert the string to a Python datetime object
#df_Transaction_unique['trans_date_trans_time'] = datetime.strptime('trans_date_trans_time', '%Y-%m-%d %H:%M:%S')

In [ ]:
df_Transaction_unique['trans_date_trans_time']

In [ ]:

# Convert datetime to string in the desired format (YYYY-MM-DD HH:MM:SS)

df_Transaction_unique['trans_date_trans_time'] = df_Transaction_unique['trans_date_trans_time'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
df_Transaction_unique['trans_date_trans_time']

In [ ]:
# Lets load the data into the MySQL database

# Establish a connection to MySQL database
try:
    connection = mysql.connector.connect(
        host='localhost',
        database='creditcard_db',
        user='root',
        password='********'
    )

    if connection.is_connected():
        print('Connected to MySQL database')

        cursor = connection.cursor()

        # Step 3: Insert data into each table
        # Insert data for Location table
        for index, row in df_Location_unique.iterrows():
            # Convert row to tuple
            cursor.execute("INSERT INTO Location (zip, city, state, latitude, longitude, city_pop) VALUES (%s, %s, %s, %s, %s, %s)",
                           tuple(row))  # Convert row to tuple

        # Insert data for Cardholder table
        for index, row in df_Cardholder_unique.iterrows():
            cursor.execute("INSERT INTO Cardholder (cc_num, first, last, gender, street, city, state, zip, job, dob) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                           tuple(row))  # Convert row to tuple

        # Insert data for Merchant table
        for index, row in df_Merchant_unique.iterrows():
            cursor.execute("INSERT INTO Merchant (merchant,category, merch_lat, merch_long) VALUES (%s, %s, %s, %s)",
                           tuple(row))  # Convert row to tuple

        # Insert data for Transaction table
        for index, row in df_Transaction_unique.iterrows():
            cursor.execute("INSERT INTO Transaction (trans_num, cc_num, trans_date_trans_time, amt, is_fraud, merchant) VALUES (%s, %s, %s, %s, %s, %s)",
                           tuple(row))  # Convert row to tuple


        # Commit the changes to the database
        connection.commit()
        print("Data inserted successfully")

except Error as e:
    print(f"Error: {e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection closed.")


